In [3]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from SALib.analyze import sobol
import warnings
from ema_workbench.analysis import feature_scoring
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    perform_experiments,
    Samplers,
    SequentialEvaluator,
)
from dike_model_function import DikeNetwork
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.analysis import prim
from ema_workbench.em_framework.parameters import Constant
import pickle
warnings.filterwarnings("ignore")

In [4]:
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [5]:
#load it
with open("data\ScenariosOpenExplo", 'rb') as file2:
    Scenarios = pickle.load(file2)

In [8]:
from ema_workbench.em_framework.optimization import ArchiveLogger, EpsilonProgress

# we need to store our results for each seed
results = []
convergences = []

with SequentialEvaluator(dike_model) as evaluator:
    # we run again for 5 seeds
    for i in range(1):
        # we create 2 covergence tracker metrics
        # the archive logger writes the archive to disk for every x nfe
        # the epsilon progress tracks during runtime
        convergence_metrics = [
            ArchiveLogger(
                r"C:\Users\roelo\Videos\archive",
                [l.name for l in dike_model.levers],
                [o.name for o in dike_model.outcomes],
                base_filename=f"{i}.tar.gz",
            ),
            EpsilonProgress(),
        ]

        result, convergence = evaluator.optimize(
            nfe=50,
            searchover="levers",
            epsilons=[0.05] * len(dike_model.outcomes),
            convergence=convergence_metrics,
            reference = Scenarios[0]
        )

        results.append(result)
        convergences.append(convergence)


  0%|                                                   | 0/50 [00:00<?, ?it/s]
100it [00:35,  2.81it/s]                                                       


In [14]:
k = r"C:\Users\roelo\Videos\archive"


'C:\\Users\\roelo\\Videos\\archive\\0.tar.gz'

In [20]:
all_archives = []

for i in range(1):
    archives = ArchiveLogger.load_archives(k+ "\\" + f"{i}.tar.gz")
    all_archives.append(archives)

In [54]:
type(archives[0])

pandas.core.frame.DataFrame

In [35]:
archives.keys()

dict_keys([0, 100])

In [43]:
del archives[100]["Unnamed: 0"]

In [55]:
del archives[0]["Unnamed: 0"]

In [52]:
del archives[100]["Unnamed: 0"]

{0: Empty DataFrame
 Columns: [Unnamed: 0, 0_RfR 0, 0_RfR 1, 0_RfR 2, 1_RfR 0, 1_RfR 1, 1_RfR 2, 2_RfR 0, 2_RfR 1, 2_RfR 2, 3_RfR 0, 3_RfR 1, 3_RfR 2, 4_RfR 0, 4_RfR 1, 4_RfR 2, EWS_DaysToThreat, A.1_DikeIncrease 0, A.1_DikeIncrease 1, A.1_DikeIncrease 2, A.2_DikeIncrease 0, A.2_DikeIncrease 1, A.2_DikeIncrease 2, A.3_DikeIncrease 0, A.3_DikeIncrease 1, A.3_DikeIncrease 2, A.4_DikeIncrease 0, A.4_DikeIncrease 1, A.4_DikeIncrease 2, A.5_DikeIncrease 0, A.5_DikeIncrease 1, A.5_DikeIncrease 2, A.1 Total Costs, A.1_Expected Number of Deaths, A.1_Expected Annual Damage, A.2 Total Costs, A.2_Expected Number of Deaths, A.2_Expected Annual Damage, A.3 Total Costs, A.3_Expected Number of Deaths, A.3_Expected Annual Damage, A.4 Total Costs, A.4_Expected Number of Deaths, A.4_Expected Annual Damage, A.5 Total Costs, A.5_Expected Number of Deaths, A.5_Expected Annual Damage, RfR Total Costs, Expected Evacuation Costs]
 Index: []
 
 [0 rows x 49 columns],
 100:     0_RfR 0  0_RfR 1  0_RfR 2  1_RfR 

In [58]:
archives

{0: Empty DataFrame
 Columns: [0_RfR 0, 0_RfR 1, 0_RfR 2, 1_RfR 0, 1_RfR 1, 1_RfR 2, 2_RfR 0, 2_RfR 1, 2_RfR 2, 3_RfR 0, 3_RfR 1, 3_RfR 2, 4_RfR 0, 4_RfR 1, 4_RfR 2, EWS_DaysToThreat, A.1_DikeIncrease 0, A.1_DikeIncrease 1, A.1_DikeIncrease 2, A.2_DikeIncrease 0, A.2_DikeIncrease 1, A.2_DikeIncrease 2, A.3_DikeIncrease 0, A.3_DikeIncrease 1, A.3_DikeIncrease 2, A.4_DikeIncrease 0, A.4_DikeIncrease 1, A.4_DikeIncrease 2, A.5_DikeIncrease 0, A.5_DikeIncrease 1, A.5_DikeIncrease 2, A.1 Total Costs, A.1_Expected Number of Deaths, A.1_Expected Annual Damage, A.2 Total Costs, A.2_Expected Number of Deaths, A.2_Expected Annual Damage, A.3 Total Costs, A.3_Expected Number of Deaths, A.3_Expected Annual Damage, A.4 Total Costs, A.4_Expected Number of Deaths, A.4_Expected Annual Damage, A.5 Total Costs, A.5_Expected Number of Deaths, A.5_Expected Annual Damage, RfR Total Costs, Expected Evacuation Costs]
 Index: []
 
 [0 rows x 48 columns],
 100:     0_RfR 0  0_RfR 1  0_RfR 2  1_RfR 0  1_RfR 1  

In [51]:
len(archives[100].columns)

48

In [18]:
from ema_workbench.em_framework.optimization import epsilon_nondominated, to_problem

In [56]:
from ema_workbench import (
    HypervolumeMetric,
    GenerationalDistanceMetric,
    EpsilonIndicatorMetric,
    InvertedGenerationalDistanceMetric,
    SpacingMetric,
)
from ema_workbench.em_framework.optimization import to_problem

problem = to_problem(dike_model, searchover="levers")

reference_set = epsilon_nondominated(archives, [0.05] * len(dike_model.outcomes), problem)

EMAError: Parameter names ['0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0', '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0', '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2', 'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2', 'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2', 'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2', 'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2', 'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2'] not found in archive

In [57]:
problem

In [22]:


hv = HypervolumeMetric(reference_set, problem)
gd = GenerationalDistanceMetric(reference_set, problem, d=1)
ei = EpsilonIndicatorMetric(reference_set, problem)
ig = InvertedGenerationalDistanceMetric(reference_set, problem, d=1)
sm = SpacingMetric(problem)


metrics_by_seed = []
for archives in archives:
    metrics = []
    for nfe, archive in archives.items():
        scores = {
            "generational_distance": gd.calculate(archive),
            "hypervolume": hv.calculate(archive),
            "epsilon_indicator": ei.calculate(archive),
            "inverted_gd": ig.calculate(archive),
            "spacing": sm.calculate(archive),
            "nfe": int(nfe),
        }
        metrics.append(scores)
    metrics = pd.DataFrame.from_dict(metrics)

    # sort metrics by number of function evaluations
    metrics.sort_values(by="nfe", inplace=True)
    metrics_by_seed.append(metrics)

EMAError: Parameter names ['0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0', '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0', '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2', 'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2', 'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2', 'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2', 'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2', 'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2'] not found in archive

In [ ]:
sns.set_style("white")
fig, axes = plt.subplots(nrows=6, figsize=(8, 12), sharex=True)

ax1, ax2, ax3, ax4, ax5, ax6 = axes

for metrics, convergence in zip(metrics_by_seed, convergences):
    ax1.plot(metrics.nfe, metrics.hypervolume)
    ax1.set_ylabel("hypervolume")

    ax2.plot(convergence.nfe, convergence.epsilon_progress)
    ax2.set_ylabel("$\epsilon$ progress")

    ax3.plot(metrics.nfe, metrics.generational_distance)
    ax3.set_ylabel("generational distance")

    ax4.plot(metrics.nfe, metrics.epsilon_indicator)
    ax4.set_ylabel("epsilon indicator")

    ax5.plot(metrics.nfe, metrics.inverted_gd)
    ax5.set_ylabel("inverted generational\ndistance")

    ax6.plot(metrics.nfe, metrics.spacing)
    ax6.set_ylabel("spacing")

ax6.set_xlabel("nfe")


sns.despine(fig)

plt.show()

### OOOOOOOOOOOOOOOOOOOOOOOOOOOO


In [ ]:
from ema_workbench import MultiprocessingEvaluator, ema_logging
from ema_workbench.em_framework.evaluators import BaseEvaluator

ema_logging.log_to_stderr(ema_logging.INFO)

with SequentialEvaluator(dike_model) as evaluator:
    results1 = evaluator.optimize(nfe=1000, searchover='levers',
                                 epsilons=[0.1,]*len(dike_model.outcomes),reference=Scenarios[0])

In [ ]:
list(dike_model.outcomes)

In [ ]:
df = pd.DataFrame(results1)

In [ ]:
df["Expected Number of Deaths"] = df["A.1_Expected Number of Deaths"] + df["A.2_Expected Number of Deaths"] + df[
    "A.3_Expected Number of Deaths"] + df["A.4_Expected Number of Deaths"] + df["A.5_Expected Number of Deaths"]
df["Expected Annual Damage"] = df["A.1_Expected Annual Damage"] + df["A.2_Expected Annual Damage"] + df[
    "A.3_Expected Annual Damage"] + df["A.4_Expected Annual Damage"] + df["A.5_Expected Annual Damage"]


## Feature importance (not required but nice to know)
df_feature_importance_out = df[["Expected Evacuation Costs", "Expected Annual Damage", "Expected Number of Deaths","A.5_Expected Number of Deaths", "A.5_Expected Annual Damage"]]

In [ ]:
from ema_workbench.analysis import parcoords

# data = results1.loc[:, [o.name for o in dike_model.outcomes]]
limits = parcoords.get_limits(df_feature_importance_out)
limits.loc[0, ["Expected Evacuation Costs","A.5_Expected Number of Deaths","A.5_Expected Annual Damage"]] = 0

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(df_feature_importance_out)
# paraxes.invert_axis('max_P')
plt.show()

In [ ]:
results1

In [ ]:
data